# Linear regression modeling

## Dataframe setup

Although our data was cleaned for the EDA section, some extra cleaning is required for running our model. Examples include, creating dummy variables as well as dropping columns with too many missing values as well as counties with missing values in any column. Below, we load in the required packages as well as clean as described above.

In [1]:
# loading in necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
# loading in obesity data from our EDA and dropping columns that have too many null values. Additonally, hawaii and alaska
# had no inputs for obesity rates which are necessary for our model. Thus, they were dropped. 
# furthermore, obesity proxies that went into creating healthy score access was also dropped.
obesity_df = pd.read_csv("../../processed_data/obesity_eda.csv")
obesity_df[~obesity_df.region.str.contains('O')]
obesity_df = obesity_df.drop(columns = ['primary_minority', 'supercenter_access_score', 'grocery_access_score', 'fullservice_access_score', 'farmersmarket_access_score', 'wic_available_per1000', 'snap_bens_per1000'])

# Loading in car access data
car_access_df = pd.read_csv("../../processed_data/car_access_2017.csv")

# Merged the two data sets
obesity_df = pd.merge(obesity_df, car_access_df, how = 'left', on = 'fips')

# made some numerical variables easier to read 
obesity_df['percent_no_car'] = obesity_df['percent_no_car'] * 100
obesity_df['pop_estimate'] = obesity_df['pop_estimate']/1000

We wanted to find the counties within the contingous US that could be described as food desserts. One definition of  'food dessert', provided by the Annie E Casie foundation can be found [here](https://www.aecf.org/blog/exploring-americas-food-deserts),and other sources present a similar idea. The food insecurity variable in our data, obtained from Feeding America and coded as the fi_rate column, was calculated using many of the factors already described in the food insecurity definition found above; the calculation can be found [here](https://www.feedingamerica.org/sites/default/files/research/map-the-meal-gap/2016/2016-map-the-meal-gap-technical-brief.pdf). What was missing was a variable for access to food. Considering we had already created a proxy for access to food (healthy_access_category which encompasses grocery stores, supercenters, restaurants, and farmers markets), we decided to use both fi_rate and healthy_access_category to determine whether or not a county was labelled as being a food dessert.

The third quartile for food insecurity is at about 15.200000 meaning 75 percent is at or below that value. We decided that any county in the highest 25% would be considered the 'highest tier' of food insecurity. We chose to take the top 25% because it is marginally conservative. We then decided too be food insecure, the county not only had to be within the top 25% of insecure counties in the United states but also have either low or medium access to healthy foods.

In [3]:
# Making our food dessert categorical variable. 
conditions = [(obesity_df["fi_rate"] > 15) & (obesity_df["healthy_access_category"] != 'high'), 
              (obesity_df["fi_rate"] <= 15)]
values = [1, 0]

obesity_df["food_dessert"] = np.select(conditions, values)

In [4]:
# dropping any remaining null values so our model can work
obesity_df = obesity_df.dropna()

In [5]:
# creating dummy variables; the VIF test does not take in categorical variables

obesity_df['region'] = obesity_df['region'].map({'N':1, 'M':2, 'S':3, 'W':4})
obesity_df['healthy_access_category'] = obesity_df['healthy_access_category'].map({'low':1, 'medium':2, 'high':3})
obesity_df['class_category'] = obesity_df['class_category'].map({'low_income':1, 'lower_mid_class':2, 'mid_class':3, 'highest_income': 4})

### Checking data to determine which variables do not have colinearity then running multivarable regression model

In order to avoid multicollinearity, [independent values which have high correlation with eachother](https://towardsdatascience.com/multi-collinearity-in-regression-fe7a2c1467ea), we used the Variance Inflation Factor (VIF) technque. In the VIF method, all features are regressed against all of the other features. We only took variables with a VIF below 10.

#### Attempt 1: Running VIF on all columns followed by model (inefficent)

In this first attempt, we took every column in our dataset and ran VIF on it. We then only put the variables with low VIF into the regression model. Following that, we removed variables one by one in the model if they did not have a significant p value.

In [6]:
# putting all of our columns into a list
indp_vars = obesity_df[['percent_pop_low_access_15',
       'percent_low_income_low_access_15', 'percent_no_car_low_access_15',
       'percent_snap_low_access_15', 'percent_child_low_access_15',
       'percent_senior_low_access_15', 'percent_white_low_access_15',
       'percent_black_low_access_15', 'percent_hispanic_low_access_15',
       'percent_asian_low_access_15', 'percent_nhna_low_access_15',
       'nhpi_low_access_15', 'percent_nhpi_low_access_15',
       'percent_multiracial_low_access_15', 'grocery_per1000', 'super_per1000',
       'convenience_per1000', 'specialty_per1000', 'snap_available_per1000',
        'farmers_markets_per1000', 'pct_fm_accepting_snap',
       'pct_fm_accept_wic', 'pct_fm_credit', 'fm_sell_frveg',
       'pct_fm_sell_frveg', 'region','cost_per_meal', 'est_annual_food_budget_shortfall',
       'school_lunch_prog_17', 'school_bfast_prog_17', 'smr_food_prog_17',
       'wic_parts_pop_17', 'fast_food_per1000', 'full_service_per1000',
       'pop_estimate', 'percent_white', 'percent_black',
       'percent_native_american', 'percent_asian', 'percent_nhpi',
       'percent_multi', 'percent_nonwhite_hispanic', 'median_household_income',
       'class_category', 'healthy_access_score',
       'healthy_access_category', 'percent_no_car', 'food_dessert']]

In [7]:
#https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/
# creating a blank dataframe followed by adding each column to a new column called feature
vif_data = pd.DataFrame()
vif_data["feature"] = indp_vars.columns

# for every feature in vif_data, run VIF on it 
vif_data["VIF"] = [variance_inflation_factor(indp_vars.values, i)
                          for i in range(len(indp_vars.columns))]

# filter the data to include VIF less than ten
vif_data = vif_data[vif_data["VIF"] < 10]

In [8]:
# code was modeled and influenced by DS4A (Correlation One) material
formula= 'obesity_rate ~  percent_no_car_low_access_15 + grocery_per1000+ super_per1000+convenience_per1000+ specialty_per1000+farmers_markets_per1000+ pct_fm_accepting_snap+pct_fm_accept_wic+ pct_fm_credit+ fm_sell_frveg+pct_fm_sell_frveg+ smr_food_prog_17+ fast_food_per1000+full_service_per1000+ percent_native_american+ percent_asian+percent_nhpi+ percent_multi+ food_dessert'
model1 = sm.ols(formula = formula, data = obesity_df)
lin_reg = model1.fit()
print(lin_reg.summary())

                            OLS Regression Results                            
Dep. Variable:           obesity_rate   R-squared:                       0.402
Model:                            OLS   Adj. R-squared:                  0.398
Method:                 Least Squares   F-statistic:                     107.5
Date:                Sat, 17 Jul 2021   Prob (F-statistic):          1.77e-321
Time:                        14:24:38   Log-Likelihood:                -8144.4
No. Observations:                3062   AIC:                         1.633e+04
Df Residuals:                    3042   BIC:                         1.645e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

#### Running VIS on columns thoght to have relevance in literature

In this second attempt, we took only columns in our dataset that had some evidence in the literature of having an effecton obesitynand ran VIF on it. We then only put the variables with low VIF into the regression model. Following that, we removed variables one by one in the model if they did not have a significant p-value.

In [9]:
# Making our food dessert categorical variable. 
indp_vars2 = obesity_df[[ 'grocery_per1000', 'super_per1000',
       'convenience_per1000', 'snap_available_per1000','fast_food_per1000', 'full_service_per1000',
       'farmers_markets_per1000', 'cost_per_meal', 'smr_food_prog_17', 'pop_estimate', 'percent_white', 'percent_black',
       'percent_native_american', 'percent_asian', 'percent_nhpi',
       'percent_multi', 'percent_nonwhite_hispanic', 'food_dessert', 'school_lunch_prog_17', 'school_bfast_prog_17']]

In [10]:
#https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/
# creating a blank dataframe followed by adding each column to a new column called feature
vif_data2 = pd.DataFrame()
vif_data2["feature"] = indp_vars2.columns

# for every feature in vif_data, run VIF on it 
vif_data2["VIF"] = [variance_inflation_factor(indp_vars2.values, i)
                          for i in range(len(indp_vars2.columns))]

# filter the data to include VIF less than ten
vif_data2 = vif_data2[vif_data2["VIF"] < 10]

In [11]:
import sklearn.model_selection as model_selection
train, test = model_selection.train_test_split(obesity_df, train_size=0.80, random_state=88)

Train = pd.DataFrame(train, columns=obesity_df.columns)
Test = pd.DataFrame(test, columns=obesity_df.columns)

In [12]:
# code was modeled and influenced by DS4A (Correlation One) material
formula2= 'obesity_rate ~ super_per1000+ convenience_per1000+fast_food_per1000+ full_service_per1000+farmers_markets_per1000+ smr_food_prog_17+ pop_estimate+percent_black+ percent_native_american+ percent_asian+percent_nhpi+ percent_multi+ percent_nonwhite_hispanic+food_dessert'
model2 = sm.ols(formula = formula2, data = Train)
lin_reg2 = model2.fit()
print(lin_reg2.summary())

                            OLS Regression Results                            
Dep. Variable:           obesity_rate   R-squared:                       0.422
Model:                            OLS   Adj. R-squared:                  0.419
Method:                 Least Squares   F-statistic:                     127.1
Date:                Sat, 17 Jul 2021   Prob (F-statistic):          2.22e-277
Time:                        14:24:38   Log-Likelihood:                -6479.8
No. Observations:                2449   AIC:                         1.299e+04
Df Residuals:                    2434   BIC:                         1.308e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

Looks like our second model would have (p = .01)
    - supercenter
    - convenience stores
    - fast food
    - full service restaurants
    - population
    - race factors (black, native, asian, hispanic) 
    - food dessert
    - population
    
    Total = 11 variables, 12 if chose a p value of .05
    
if it was .05 then only addition is summer food programs

## Fitting and testing models with Sklearn

Sklearn also allows us to create our regression model. It gives less statistical information in comparison to statsmodels; however, it has several methods of testing the accuracy of our model. Understanding and creation of the code below is in large part a thanks to the articles written by [Scott Robinson](https://stackabuse.com/linear-regression-in-python-with-scikit-learn) and [Deepika Singh](https://www.pluralsight.com/guides/validating-machine-learning-models-scikit-learn)

In [13]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

### Model 1 with sklearn

In [14]:
obesity_df2 = obesity_df.copy().dropna()
obesity_df2 = obesity_df2[['super_per1000', 'convenience_per1000','fast_food_per1000', 'full_service_per1000','farmers_markets_per1000', 'smr_food_prog_17', 'pop_estimate','percent_black', 'percent_native_american', 'percent_asian','percent_nhpi', 'percent_multi', 'percent_nonwhite_hispanic','food_dessert', 'obesity_rate']]
x1 = obesity_df2.drop('obesity_rate', axis=1).values 
y1 = obesity_df2['obesity_rate'].values

In [15]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(x1, y1, test_size=0.20, random_state=100)
model = LinearRegression()
model.fit(X_train, Y_train)
result = model.score(X_test, Y_test)
print("Accuracy: %.2f%%" % (result*100.0))

Accuracy: 45.75%


In [16]:
kfold = model_selection.KFold(n_splits= 3)
model_kfold = LinearRegression()
results_kfold = model_selection.cross_val_score(model_kfold, x1, y1, cv=kfold)
print("Accuracy: %.2f%%" % (results_kfold.mean()*100.0)) 

Accuracy: 34.10%


In [17]:
y_pred = model.predict(X_test)
df = pd.DataFrame({'Actual': Y_test, 'Predicted': y_pred})
df['off'] = df['Actual'] - df['Predicted']
df.describe()

,Actual,Predicted,off
count,613.000000,613.000000,613.000000
mean,31.255465,31.026476,0.228989
std,4.544715,2.644108,3.339614
min,16.400000,20.933228,-12.104994
25%,28.800000,29.643227,-1.965399
50%,31.200000,30.912940,0.570538
75%,33.900000,32.134648,2.512459
max,46.600000,40.262339,8.329002


### Model 2 with Sklearn

In [18]:
obesity_df3 = obesity_df.copy()
obesity_df3 = obesity_df3[['grocery_per1000', 'super_per1000','obesity_rate',
       'convenience_per1000', 'snap_available_per1000','fast_food_per1000', 'full_service_per1000',
       'farmers_markets_per1000', 'cost_per_meal', 'smr_food_prog_17', 'pop_estimate', 'percent_white', 'percent_black',
       'percent_native_american', 'percent_asian', 'percent_nhpi',
       'percent_multi', 'percent_nonwhite_hispanic', 'food_dessert', 'school_lunch_prog_17', 'school_bfast_prog_17']]


In [19]:
x2 = obesity_df3.drop('obesity_rate', axis=1).values 
y2 = obesity_df3['obesity_rate'].values

X2_train, X2_test, Y2_train, Y2_test = model_selection.train_test_split(x2, y2, test_size=0.20, random_state=100)
model = LinearRegression()
model.fit(X2_train, Y2_train)
result = model.score(X2_test, Y2_test)
print("Accuracy: %.2f%%" % (result*100.0))

Accuracy: 55.81%


In [20]:
kfold = model_selection.KFold(n_splits= 3)
model_kfold = LinearRegression()
results_kfold = model_selection.cross_val_score(model_kfold, x2, y2, cv=kfold)
print("Accuracy: %.2f%%" % (results_kfold.mean()*100.0)) 

Accuracy: 49.26%


In [21]:
y2_pred = model.predict(X2_test)
df2 = pd.DataFrame({'Actual': Y2_test, 'Predicted': y2_pred})
df2['off'] = df2['Actual'] - df2['Predicted']
df2.describe()

,Actual,Predicted,off
count,613.000000,613.000000,613.000000
mean,31.255465,31.029891,0.225574
std,4.544715,3.105654,3.012616
min,16.400000,19.881809,-10.668817
25%,28.800000,29.202480,-1.682193
50%,31.200000,31.174336,0.375088
75%,33.900000,32.602155,2.344844
max,46.600000,41.957531,8.498632


### Model 3 with sklearn

In [22]:
obesity_df4 = obesity_df.copy()
obesity_df4 = obesity_df4[['percent_no_car_low_access_15', 'percent_hispanic_low_access_15',
       'percent_asian_low_access_15', 'nhpi_low_access_15',
       'percent_nhpi_low_access_15', 'grocery_per1000', 'super_per1000',
       'convenience_per1000', 'specialty_per1000',
       'farmers_markets_per1000', 'pct_fm_accepting_snap',
       'pct_fm_accept_wic', 'pct_fm_credit', 'fm_sell_frveg',
       'pct_fm_sell_frveg', 'smr_food_prog_17', 'fast_food_per1000',
       'full_service_per1000', 'percent_native_american', 'percent_asian',
       'percent_nhpi', 'percent_multi', 'food_dessert', 'obesity_rate']]


In [23]:
x3 = obesity_df4.drop('obesity_rate', axis=1).values 
y3 = obesity_df4['obesity_rate'].values

X3_train, X3_test, Y3_train, Y3_test = model_selection.train_test_split(x3, y3, test_size=0.20, random_state=100)
model = LinearRegression()
model.fit(X3_train, Y3_train)
result = model.score(X3_test, Y3_test)
print("Accuracy: %.2f%%" % (result*100.0))

Accuracy: 42.00%


In [24]:
kfold = model_selection.KFold(n_splits= 3)
model_kfold = LinearRegression()
results_kfold = model_selection.cross_val_score(model_kfold, x3, y3, cv=kfold)
print("Accuracy: %.2f%%" % (results_kfold.mean()*100.0)) 

Accuracy: 34.25%


In [25]:
y3_pred = model.predict(X3_test)
df3 = pd.DataFrame({'Actual': Y3_test, 'Predicted': y3_pred})
df3['off'] = df3['Actual'] - df3['Predicted']
df3.describe()

,Actual,Predicted,off
count,613.000000,613.000000,613.000000
mean,31.255465,31.026496,0.228969
std,4.544715,2.691547,3.453515
min,16.400000,18.932068,-11.543924
25%,28.800000,29.427124,-1.735934
50%,31.200000,31.053945,0.355401
75%,33.900000,32.708653,2.517341
max,46.600000,40.479360,12.475967


## Predicting obesity

#### based on model two from statsmodels

In [26]:
import statsmodels.api as sm

Here we used the second linear regression model (found above) to predict obesity rate based on new input variables.

In [27]:
# These are the coefficents of the variables in the model
lin_reg2.params

Intercept                    32.144153
super_per1000                16.743927
convenience_per1000           1.886535
fast_food_per1000            -0.623807
full_service_per1000         -2.385076
farmers_markets_per1000      -0.399200
smr_food_prog_17             -0.031962
pop_estimate                 -0.001051
percent_black                 0.080931
percent_native_american       0.062605
percent_asian                -0.443510
percent_nhpi                 -0.500720
percent_multi                -0.283441
percent_nonwhite_hispanic    -0.045263
food_dessert                  0.782807
dtype: float64

In [28]:
obesity_df

,fips,state,county,percent_pop_low_access_15,percent_low_income_low_access_15,percent_no_car_low_access_15,percent_snap_low_access_15,percent_child_low_access_15,percent_senior_low_access_15,percent_white_low_access_15,...,percent_nhpi,percent_multi,percent_nonwhite_hispanic,median_household_income,obesity_category,class_category,healthy_access_score,healthy_access_category,percent_no_car,food_dessert
0,1001,AL,Autauga,0.32,0.12,0.03,0.05,0.08,0.04,0.23,...,0.1,0.4,2.7,58343.0,high,3,7,2,5.41,0
1,1003,AL,Baldwin,0.17,0.05,0.02,0.01,0.04,0.03,0.14,...,0.0,0.4,4.4,56607.0,med,3,9,2,3.35,0
2,1005,AL,Barbour,0.22,0.11,0.04,0.04,0.04,0.03,0.10,...,0.0,0.3,4.2,32490.0,high,1,9,2,9.63,1
3,1007,AL,Bibb,0.04,0.03,0.03,0.01,0.01,0.01,0.02,...,0.0,0.5,2.4,45795.0,high,2,8,2,5.70,0
4,1009,AL,Blount,0.06,0.03,0.03,0.01,0.02,0.01,0.06,...,0.0,0.4,9.0,48253.0,high,2,6,1,4.14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3097,56037,WY,Sweetwater,0.43,0.11,0.02,0.02,0.11,0.04,0.38,...,0.5,0.6,16.0,75590.0,med,3,7,2,2.97,0
3098,56039,WY,Teton,0.29,0.07,0.01,0.01,0.05,0.03,0.27,...,0.0,0.6,15.0,90145.0,low,3,10,3,1.65,0
3099,56041,WY,Uinta,0.22,0.10,0.03,0.02,0.07,0.02,0.20,...,0.0,0.9,9.1,67404.0,med,3,10,3,4.21,0
3100,56043,WY,Washakie,0.11,0.04,0.01,0.01,0.02,0.03,0.10,...,0.0,1.1,14.2,57989.0,med,3,10,3,5.85,0


In [29]:
# Here we ask the user for new input variables. However, we do not allow racial makeup to be changed due to 
# negative implications. 

fips = float(input())
super_input = float(input())
convenience_store = float(input())
fast_food = float(input())
restaurant = float(input())
farmers_market = float(input())
summer_prog = float(input())
population = obesity_df.loc[obesity_df['fips'] == fips, 'pop_estimate'].iloc[0]
black = obesity_df.loc[obesity_df['fips'] == fips, 'percent_black'].iloc[0]
native = obesity_df.loc[obesity_df['fips'] == fips, 'percent_native_american'].iloc[0]
asian = obesity_df.loc[obesity_df['fips'] == fips, 'percent_asian'].iloc[0]
nhpi = obesity_df.loc[obesity_df['fips'] == fips, 'percent_nhpi'].iloc[0]
multi = obesity_df.loc[obesity_df['fips'] == fips, 'percent_multi'].iloc[0]
hisp = obesity_df.loc[obesity_df['fips'] == fips, 'percent_nonwhite_hispanic'].iloc[0]
food_des = obesity_df.loc[obesity_df['fips'] == fips, 'food_dessert'].iloc[0]

1001
1
1
1
1
1
1


In [30]:
# we then create a new dataframe with the variables above so we can input it into the model prediction
new_vals = pd.DataFrame({'super_per1000': [super_input], 'convenience_per1000': [convenience_store],'fast_food_per1000': [fast_food], 'full_service_per1000': [restaurant], 'farmers_markets_per1000': [farmers_market], 'smr_food_prog_17': [summer_prog], 'pop_estimate': [population], 'percent_black': [black],'percent_native_american': [native],'percent_asian': [asian], 'percent_nhpi': [nhpi], 'percent_multi': [multi], 'percent_nonwhite_hispanic': [hisp], 'food_dessert': [food_des]})

In [32]:
# the new variables are put into the new model and then the second value is the predicted obesity in the county based upon changes
xnew = sm.add_constant(new_vals)
ynewpred =  lin_reg2.predict(xnew)
ynewpred

0    48.156464
dtype: float64